In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


土地数値情報のZIP内のファイル数を確認する

In [ ]:
import zipfile

# ZIPファイルのパス
zip_path = '/content/drive/My Drive/signate/L02-24_GML.zip'

# ZIPファイル内のファイル一覧を取得
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()  # ZIP内のファイル名一覧を取得

# ファイル一覧を表示
print("Files in ZIP:")
for file in file_list:
    print(file)

# ZIP内のファイル数を表示
print(f"\nNumber of files in ZIP: {len(file_list)}")


Files in ZIP:
KS-META-L02-24.xml
L02-24.dbf
L02-24.geojson
L02-24.prj
L02-24.shp
L02-24.shx
L02-24.xml

Number of files in ZIP: 7


土地数値情報のZIPファイルを解凍する

In [ ]:
import zipfile
import os


# train_ZIPファイルのパス
zip_path = '/content/drive/My Drive/signate/L02-24_GML.zip'
extract_path = '/content/drive/My Drive/signate/extracted_GML'

# 解凍
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# 解凍されたファイルを確認
print("Extracted files:", os.listdir(extract_path))

Extracted files: ['KS-META-L02-24.xml', 'L02-24.dbf', 'L02-24.geojson', 'L02-24.prj', 'L02-24.shp', 'L02-24.shx', 'L02-24.xml']


ファイル概要

1. KS-META-L02-24.xml: メタデータファイル。データの概要や構造が記載されていることが多い。

2. L02-24.dbf: 属性情報を格納したデータベースファイル。これが**主に属性データ（例えば用途地域や都市計画など）**を含む重要なファイルです。

3. L02-24.geojson: 地理情報を含むJSON形式のファイル。地理データを扱いやすい形式に変換済み。

4. L02-24.prj: 投影情報ファイル。地図の投影方式が記載されています。

5. L02-24.shp: 空間データ（形状情報）を格納したファイル。

6. L02-24.shx: SHPファイルのインデックスファイル。

7. L02-24.xml: データに関する追加情報を含むXMLファイル。


KS-META-L02-24.xmlを確認する

In [ ]:
!pip install xmltodict


エンコーディングを確認する

In [ ]:
import chardet

# ファイルのエンコーディングを推定
file_path = '/content/drive/My Drive/signate/extracted_GML/KS-META-L02-24.xml'

with open(file_path, 'rb') as file:
    raw_data = file.read(10000)  # 最初の10,000バイトを確認
    result = chardet.detect(raw_data)
    print(f"推定エンコーディング: {result['encoding']}")


推定エンコーディング: SHIFT_JIS


In [ ]:
#パンダスと一緒に使えるライブラリ
import xmltodict

# XMLファイルの読み込み
file_path = '/content/drive/My Drive/signate/extracted_GML/KS-META-L02-24.xml'

# エンコーディングを Shift_JISに指定
with open(file_path, 'r', encoding='shift_jis') as file:
    xml_content = file.read()

# XMLを辞書に変換
data_dict = xmltodict.parse(xml_content)

# 上位キーを表示
print(data_dict.keys())


dict_keys(['MD_Metadata'])


L02-24.dbfファイルを確認する

In [ ]:
import geopandas as gpd

# DBFファイルの読み込み
dbf_path = '/content/drive/My Drive/signate/extracted_GML/L02-24.dbf'
dbf = gpd.read_file(dbf_path)

# 上位5行を表示
print(dbf.head())


  L02_001 L02_002 L02_003 L02_004  L02_005  L02_006  L02_007  L02_008 L02_009  \
0     000     001     000     001     2024   252000      2.9        1   false   
1     000     002     000     002     2024   230000      2.7        1   false   
2     000     003     000     003     2024   200000      2.6        1   false   
3     000     004     000     004     2024    67000      3.1        1   false   
4     000     005     000     005     2024   375000      2.2        1   false   

  L02_010  ...         L02_129         L02_130         L02_131  \
0   false  ...  10000000000000  10000000000000  10000000000000   
1   false  ...  40000000000000  10000000000000  10000000000000   
2   false  ...  10000000000000  10000000000000  10000000000000   
3   false  ...  10000000000000  10000000000000  10000000000000   
4   false  ...  10000010000000  10000000000000  10000000000000   

          L02_132         L02_133       L02_134       L02_135       L02_136  \
0  10000000000000  10000000000000  10

使用するデータ

1.   L02_005: 年度（対象とする地価調査の年度）
2.   L02_006: 調査価格
3.   L02_007: 対前年変動率（当年価格- 前年価格）* 100 / 前年価格
4.   L02_020: 基準値行政地区コード
5.   L02_021: 市区町村を示す地名
6.   L02_022: 基準値の所在および地番
7.   L02_023: 基準値が住居表示を実施している場合の住居表示住所（町名以降）
8.   L02_028: 建物構造
9.   L02_044: 基準値の最寄り駅名
10.  L02_045: 基準値から最寄駅までの道路距離
11.  L02_046: 都市計画の用途地域
12.  L02_050: 建蔽率
13.  L02_051: 容積率
14.  geometry : 緯度経度

現在の列名	新しい列名	理由
1. L02_005	target_year	年度（対象とする地価調査の年度）
2. L02_006	price_survey	調査価格
3. L02_007	price_change_rate	対前年変動率
4. L02_020	admin_area_code	行政地区コード
5. L02_021	addr1_1	市区町村名
6. L02_022	land_address	地番
7. L02_023	residence_address	住居表示住所
8. L02_028	building_structure	建物構造
9. L02_044	eki_name1	最寄り駅
10. L02_045: walk_distance1 基準値から最寄駅までの道路距離
11. L02_046	land_youto	用途地域
12. L02_046	land_toshi	都市計画
12. L02_050	land_kenpei	建蔽率
13. L02_051	land_youseki	容積率
14. geometry	geometry	緯度経度情報（変更なし）

In [ ]:
import geopandas as gpd

# DBFファイルのパス
dbf_path = '/content/drive/My Drive/signate/extracted_GML/L02-24.dbf'

# 必要な列を選択して読み込む
columns_to_select = [
    'L02_005', 'L02_006', 'L02_007', 'L02_020', 'L02_021',
    'L02_022', 'L02_023', 'L02_044', 'L02_045',
    'L02_046', 'L02_048', 'L02_050', 'L02_051', 'geometry'
]

dbf = gpd.read_file(dbf_path)[columns_to_select]

# 列名を変更
column_rename_map = {
    'L02_005': 'target_year',
    'L02_006': 'price_2024',
    'L02_007': 'price_change_rate',
    'L02_020': 'admin_area_code',
    'L02_021': 'addr1_1',
    'L02_022': 'land_address',
    'L02_023': 'residence_address',
    'L02_044': 'eki_name1',
    'L02_045': 'walk_distance1',
    'L02_046': 'land_youto',
    'L02_048': 'land_toshi',
    'L02_050': 'land_kenpei',
    'L02_051': 'land_youseki',
    'geometry': 'geometry'
}

dbf.rename(columns=column_rename_map, inplace=True)

# データの形状（行数と列数）
print(f"Rows: {dbf.shape[0]}, Columns: {dbf.shape[1]}")

# 確認
print(dbf.head())

Rows: 21425, Columns: 14
   target_year  price_2024  price_change_rate admin_area_code addr1_1  \
0         2024      252000                2.9           01101    札幌中央   
1         2024      230000                2.7           01101    札幌中央   
2         2024      200000                2.6           01101    札幌中央   
3         2024       67000                3.1           01101    札幌中央   
4         2024      375000                2.2           01101    札幌中央   

                land_address residence_address eki_name1  walk_distance1  \
0      北海道　札幌市中央区宮の森３条５丁目８番１       宮の森３条５－１－３１  地下鉄西２８丁目             650   
1  北海道　札幌市中央区南１８条西８丁目５９３番１８外       南１８条西８－１－４３    地下鉄幌平橋             800   
2   北海道　札幌市中央区南２２条西８丁目６３０番４外       南２２条西８－３－２６    地下鉄幌平橋            1600   
3    北海道　札幌市中央区界川２丁目１８８０番４１０          界川２－４－２４   地下鉄円山公園            2500   
4     北海道　札幌市中央区宮ヶ丘２丁目４７４番８６         宮ヶ丘２－１－２０  地下鉄西２８丁目             500   

  land_youto land_toshi  land_kenpei  land_youseki                    geometry 

In [ ]:
# land_youto 列のユニークな値を取得
unique_land_youto = dbf['land_youto'].unique()

# ユニークな値を確認
print("ユニークな用途地域:")
print(unique_land_youto)
# ユニークな値の数を確認
num_unique_land_youto = dbf['land_youto'].nunique()

print(f"用途地域のユニークな値の数: {num_unique_land_youto}")


ユニークな用途地域:
['1中専' '1住居' '2中専' '1低専' '商業' '近商' '準工' '2住居' '2低専' '工専' '_' '工業' '準住居']
用途地域のユニークな値の数: 13


 trainデータに合わせてユニークな値を変更する

1. '1低専': 1
2. '2中専': 2
3. '2住居' : 3
4. '近商': 4
5. '商業': 5
6. '準工': 6
7. '工業' : 7
8. '工専': 8
9. '2低専' 10
10. '2低専' 10
11. '1中専' :11
12. '1住居': 12
13. '準住居': 13
14. '_': 99

In [ ]:
from pandas.api.types import CategoricalDtype

# マッピング辞書を作成
land_youto_mapping = {
    '1低専': 1,
    '2中専': 2,
    '2住居': 3,
    '近商': 4,
    '商業': 5,
    '準工': 6,
    '工業': 7,
    '工専': 8,
    '2低専': 10,
    '1中専': 11,
    '1住居': 12,
    '準住居': 13,
    '_': 99
}

# land_youto 列をマッピング
dbf['land_youto'] = dbf['land_youto'].replace(land_youto_mapping).infer_objects()

# カテゴリ型に変換
dbf['land_youto'] = dbf['land_youto'].astype('category')

# 確認
print(dbf['land_youto'].dtypes)  # カテゴリ型の確認
print(dbf['land_youto'].cat.categories)  # カテゴリの一覧


category
Index([1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 99], dtype='int64')


<ipython-input-77-b867e8d98422>:21: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dbf['land_youto'] = dbf['land_youto'].replace(land_youto_mapping).infer_objects()


land_toshiのユニークな値をtrainデータの値と合わせる

In [ ]:
# land_toshi列のユニークな値を取得
unique_land_toshi = dbf['land_toshi'].unique()

# ユニークな値を確認
print("ユニークな用途地域:")
print(unique_land_toshi)
# ユニークな値の数を確認
num_unique_land_toshi = dbf['land_toshi'].nunique()

print(f"都市計画のユニークな値の数: {num_unique_land_toshi}")


ユニークな用途地域:
['市街化' '都計外' '調区' '非線引' '準都計']
都市計画のユニークな値の数: 5


In [ ]:
# 欠損値の処理と型の統一
dbf['land_toshi'] = dbf['land_toshi'].fillna('その他').astype(str)

# マッピング辞書を適用
land_toshi_mapping = {
    '市街化': 1,
    '調区': 2,
    '非線引': 3,
    '都計外': 4,
    '準都計': 5,
    'その他': 99  # デフォルト値
}
dbf['land_toshi'] = dbf['land_toshi'].replace(land_toshi_mapping).infer_objects()

# カテゴリ型に変換
dbf['land_toshi'] = dbf['land_toshi'].astype('category')

# 確認
print(dbf['land_toshi'].dtypes)  # カテゴリ型の確認
print(dbf['land_toshi'].cat.categories)  # カテゴリの一覧


category
Index([1, 2, 3, 4, 5], dtype='int64')


<ipython-input-79-b9f72762a33d>:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dbf['land_toshi'] = dbf['land_toshi'].replace(land_toshi_mapping).infer_objects()


In [ ]:
import os

# 保存先のGoogle Driveフォルダのパス
folder_path = "/content/drive/My Drive/signate/extracted_GML/"

# フォルダが存在しない場合は作成
os.makedirs(folder_path, exist_ok=True)

# 保存先のファイルパス
output_file_path = os.path.join(folder_path, "selected_dbf.csv")

# データをCSVとして保存
dbf.to_csv(output_file_path, index=False)



print(f"データが保存されました: {output_file_path}")

データが保存されました: /content/drive/My Drive/signate/extracted_GML/selected_dbf.csv


In [ ]:
!pip install shapely


In [ ]:
import pandas as pd

dbf_path = '/content/drive/My Drive/signate/extracted_GML/selected_dbf.csv'

# CSVデータを読み込む
dbf_data = pd.read_csv(dbf_path, low_memory=False)

# geometry 列のデータ型を確認
print(f"geometry 列のデータ型: {dbf_data['geometry'].dtype}")


geometry 列のデータ型: object


In [ ]:
import pandas as pd
from shapely.wkt import loads

# geometry 列を shapely オブジェクトに変換
dbf_data['geometry'] = dbf_data['geometry'].apply(loads)

# lon（経度）と lat（緯度）を抽出して新しい列に追加
dbf_data['lon'] = dbf_data['geometry'].apply(lambda geom: geom.x)
dbf_data['lat'] = dbf_data['geometry'].apply(lambda geom: geom.y)

# 確認
print(dbf_data[['geometry', 'lon', 'lat']].head())


                                       geometry         lon        lat
0   POINT (141.3072936000001 43.06356833000007)  141.307294  43.063568
1  POINT (141.34943690000011 43.03638694000006)  141.349437  43.036387
2  POINT (141.34863670000004 43.02999028000005)  141.348637  43.029990
3   POINT (141.3158281000001 43.03567194000004)  141.315828  43.035672
4  POINT (141.31079810000006 43.05838667000006)  141.310798  43.058387


In [ ]:
# geometry, addr1_1 列を削除
dbf_data = dbf_data.drop(columns=['geometry', 'addr1_1'])

# selected_dbf の列名を full_address に変更
dbf_data = dbf_data.rename(columns={'residence_address': 'full_address'})

# selected_dbf の列名を full_address に変更
dbf_data = dbf_data.rename(columns={'full_address': 'sub_address'})

# admin_area_code を area_JIS_code に変更
dbf_data = dbf_data.rename(columns={'admin_area_code': 'area_JIS_code'})

# 変更後の列名
dbf_data['area_JIS_code'] = dbf_data['area_JIS_code'].apply(lambda x: f"{int(x):05d}")

# カテゴリ型に変換
dbf_data['area_JIS_code'] = dbf_data['area_JIS_code'].astype('category')


# 確認
print(dbf_data.head())


   target_year  price_2024  price_change_rate area_JIS_code  \
0         2024      252000                2.9         01101   
1         2024      230000                2.7         01101   
2         2024      200000                2.6         01101   
3         2024       67000                3.1         01101   
4         2024      375000                2.2         01101   

                land_address  sub_address eki_name1  walk_distance1  \
0      北海道　札幌市中央区宮の森３条５丁目８番１  宮の森３条５－１－３１  地下鉄西２８丁目             650   
1  北海道　札幌市中央区南１８条西８丁目５９３番１８外  南１８条西８－１－４３    地下鉄幌平橋             800   
2   北海道　札幌市中央区南２２条西８丁目６３０番４外  南２２条西８－３－２６    地下鉄幌平橋            1600   
3    北海道　札幌市中央区界川２丁目１８８０番４１０     界川２－４－２４   地下鉄円山公園            2500   
4     北海道　札幌市中央区宮ヶ丘２丁目４７４番８６    宮ヶ丘２－１－２０  地下鉄西２８丁目             500   

   land_youto  land_toshi  land_kenpei  land_youseki         lon        lat  
0          11           1           60           200  141.307294  43.063568  
1          12           1           60

In [ ]:
import os

# 保存先のGoogle Driveフォルダのパス
folder_path = "/content/drive/My Drive/signate/extracted_GML/"

# フォルダが存在しない場合は作成
os.makedirs(folder_path, exist_ok=True)

# 保存先のファイルパス
output_file_path = os.path.join(folder_path, "edit_dbf.csv")

# データをCSVとして保存
dbf_data.to_csv(output_file_path, index=False)

print(f"データが保存されました: {output_file_path}")


データが保存されました: /content/drive/My Drive/signate/extracted_GML/edit_dbf.csv
